In [1]:
import pandas as pd
from seml import get_results

In [ ]:
df = get_results(
    'dp_timeseries_dp_train_standard_eval',
    [
        'config.seed',
        'config.estimator_name',
        'config.estimator_kwargs.batch_size',
        'config.dp_optimizer_kwargs.max_grad_norm',
        'config.dp_optimizer_kwargs.noise_multiplier',
        'config.estimator_kwargs.relative_context_length',
        'config.dp_accountant_kwargs.budget_epsilon',
        'result.metrics_test.mean_wQuantileLoss',
        'result.metrics_test.MASE'
    ],
    to_data_frame=True,
    filter_dict={
        'config.dataset_kwargs.dataset_name': 'traffic',
        'config.estimator_kwargs.trainer_kwargs.max_epochs': 4000,
        'config.top_level_mode': 'sampling_without_replacement',
        'config.instances_per_sequence': 1,
        'config.estimator_kwargs.batch_size': {'$in': [128, 256]},
        'config.dp_optimizer_kwargs.max_grad_norm': {'$in': [0.001, 0.0001]},
        'config.dp_optimizer_kwargs.noise_multiplier': {'$in': [2.0, 4.0]},
        'config.estimator_kwargs.relative_context_length': {'$in': [8]},
        'config.dp_accountant_kwargs.budget_delta': 1e-7,
        'config.dp_accountant_kwargs.budget_epsilon': {'$in': [0.25, 0.5, 1.0, 2.0, 4.0, 8.0]},
        'config.estimator_kwargs.lags_seq': {'$in': [None, [
            1, 2, 3, 4, 5, 6, 7, 23, 24, 25, 47, 48, 49, 71, 72, 73,
            95, 96, 97, 119, 120, 121, 143, 144, 145, 167, 168, 169]]},
        'config.tight_privacy_loss': True
    }
)

In [3]:
columns = df.columns[1:]

df = df.rename(columns={
    c: c.split('.')[-1]
    for c in columns
})

In [ ]:
df

In [5]:
def create_table(df: pd.DataFrame,
                 batch_size: int,
                 max_grad_norm: float,
                 noise_multiplier: float,
                 errors: bool = False
) -> pd.DataFrame:

    df = df.copy()

    df = df[
        (df['batch_size'] == batch_size)
        & (df['max_grad_norm'] == max_grad_norm)
        & (df['noise_multiplier'] == noise_multiplier)
    ]

    print(len(df))

    # Average over random seeds
    if not errors:
        df = df.groupby(
            ['max_grad_norm', 'noise_multiplier', 'batch_size', 'estimator_name', 'budget_epsilon']
        ).mean().reset_index()
    else:
        df = df.groupby(
            ['max_grad_norm', 'noise_multiplier', 'batch_size', 'estimator_name', 'budget_epsilon']
        ).std().reset_index()

    df = df.drop(columns=[
        '_id',
        'seed', 'relative_context_length',
        'max_grad_norm', 'noise_multiplier', 'batch_size', '_id'])

    df = df.sort_values(by=['estimator_name', 'budget_epsilon'])

    return df

### Batch size 256, Noise Multiplier 4.0

In [ ]:
df_mean = create_table(df, 256, 0.0001, 4.0)
df_mean

In [ ]:
df_std = create_table(df, 256, 0.0001, 4.0, errors=True)
df_std

In [ ]:
i = 0
names = ['DLinear', 'DeepAR', 'iTransf.', 'SimpleFF']

acc = f'{names[0]} &'

for i, (mean, std) in enumerate(zip(df_mean['mean_wQuantileLoss'], df_std['mean_wQuantileLoss'])):

    if (i > 0) and ((i % 6)  == 0):
        acc += ' ? \\\\'
        print(acc)
        acc = f'{names[i // 6]} &'
    acc += ' $' + f'{mean:.3f}' + '$ '
    acc += '\\tiny{$\\pm ' + f'{std:.3f}' + '$} &'

acc += ' ? \\\\ '
print(acc)